In [ ]:
# Created to run on Google Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import math
import pandas as pd

import zipfile

In [ ]:
# Update the file paths, names, etc

path = 'drive/MyDrive/Data/'
file_name_1 = "video_list.txt"

# Load the video list
video_list = pd.read_csv(path+file_name_1,delimiter="\t")

# Zip file which will contain the frames and the TSV file which will contain the labels
zip_name  = 'day_7.zip'
txt_name = "frames_day_7.txt"

# Path to videos in Google Drive and an example video name
vid_path = 'drive/MyDrive/Data/lol_worlds_2020_groups/xxx/'
file_name_0 = "TSM vs GEN _ Worlds Group Stage Day 7 _ Team SoloMid vs Gen.G (2020).mp4"


cap = cv2.VideoCapture(vid_path+file_name_0)
if (cap.isOpened()== False):
  print("Error opening video  file")

frame_rate = cap.get(5)
print(frame_rate)

In [ ]:
# Get video id for current video
id = video_list[(video_list.title == file_name_0)].id.values
id = str(id[0])
print(id)

In [ ]:
# create zip if zip file !exist in drive
zf = zipfile.ZipFile(path+zip_name, mode='w')
zf.close()

In [ ]:
# If zip exists in drive
zf = zipfile.ZipFile(path+zip_name, mode='a')

In [ ]:
count = 0
frame_id = cap.get(1) 
frames = []
labels = []

# Read until video is completed
while(cap.isOpened()):
  frame_id = cap.get(1) 

  ret, frame = cap.read()
  if ret != True:
    print("Exit", frame_id)
    break

  if frame_id % math.floor(frame_rate * 0.5) == 0 :
    # resize to 720p
    frame1 = cv2.resize(frame,(1280,720),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)

    frame_name = id+"_%d.jpg" % frame_id
    # filename = path + "frames_1/"

    cv2.imwrite("frame/"+frame_name, frame1)
    try:
        zf.write("frame/"+frame_name)
    except:
        print("Zip Fail!")

    count+=1
    print("Saved: ", count)

    frames.append(frame_name)
    # All frames are labeled a default 0
    labels.append(0)

# release the video capture object
cap.release()

#close zip
zf.close()

df = pd.DataFrame()
df['frame'] = frames
df['label'] = labels

# New file
# df.to_csv(path+txt_name, sep ='\t', index=False)

# Append to file
df.to_csv(path+txt_name, sep ='\t', index=False, mode='a', header=False)

In [ ]:
!rm -rf frame